# Рекомендация тарифов

## Откройте и изучите файл

In [1]:
# Импортируем необходимые библиотеки для работы (буду добавлять в эту ячейку нужные по мере необходимости)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Читаем файл, смотрим первые 5 строк для изучения
df = pd.read_csv('/workspaces/test_project/ML&Data/users_behavior.csv')
print(df.head(5))
print()
print(df.info())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


In [3]:
# Переведем данные о звонках и сообщениях к типу Int
df['calls']= df['calls'].astype('Int64')
df['messages']= df['messages'].astype('Int64')

Поскольку мы решаем задачу классификации, то мы можем рассмотреть следующие модели:
- Модель дерева решений
- Модель случайного леса
- Модель логистической регрессии

## Разбиение данных на выборки

Данные будут разбиты на выборки в следующем соотношении:
- Учебная - 60%
- Тестовая - 20%
- Валидационная - 20%

In [4]:
# Выделим признаки
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [5]:
# Выделим учебную выборку
features_train, features_test, target_train, target_test = train_test_split(features,target, test_size=.4, random_state=12345)

In [6]:
# Выделим тестовую и валидационную выборки (поделим тестовую из скрипта выше в соотношении 50/50)
features_valid, features_test, target_valid, target_test = train_test_split(features_test,target_test, test_size=0.5, random_state=12345) 

In [7]:
# Посмотрим на размеры выборок
# Обучающая
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


In [8]:
# Тестовая
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


In [9]:
# Валидационная
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


## Исследование модели

### Модель дерева решений

In [10]:
# Построим модель дерева решений, изменяя параметр глубины дерева от 1 до 10 
#с целью нахождения модели с максимальным показателем accuracy
best_model_tree = None
best_depth_tree = 0
best_accuracy_tree = 0
for depth in range(1,10):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth).fit(features_train,target_train)
    predictions_tree = model_tree.predict(features_valid)
    score_tree = accuracy_score(target_valid,predictions_tree)
    if best_accuracy_tree<score_tree:
        best_model_tree = model_tree
        best_depth_tree = depth
        best_accuracy_tree = score_tree

In [11]:
# Выведем данные о лучшей модели
print(f'Лучший показатель depth: {best_depth_tree}, при нем показатель accuracy равен: {best_accuracy_tree}')

Лучший показатель depth: 3, при нем показатель accuracy равен: 0.7853810264385692


### Модель случайного леса

In [12]:
# Построим модель случайного леса, изменяя параметр глубины дерева и количество деревьев от 1 до 10 
#с целью нахождения модели с максимальным показателем accuracy
best_model_forest = None
best_n_forest = 0
best_depth_forest = 0
best_accuracy_forest = 0
for n in range(1,11):
    for depth in range(1,11):
        model_forest = RandomForestClassifier(random_state=12345,n_estimators=n,max_depth=depth).fit(features_train,target_train)
        predictions_forest = model_forest.predict(features_valid)
        score_forest = accuracy_score(target_valid,predictions_forest)
        if best_accuracy_forest < score_forest:
            best_model_forest = model_forest
            best_n_forest = n
            best_depth_forest = depth
            best_accuracy_forest = score_forest

In [13]:
# Выведем данные о лучшей модели
print(f'Лучший показатель depth: {best_depth_forest} с количество деревьев {best_n_forest}, при нем показатель accuracy равен: {best_accuracy_forest}')

Лучший показатель depth: 8 с количество деревьев 8, при нем показатель accuracy равен: 0.80248833592535


### Модель логистической регрессии

In [14]:
model_regression = LogisticRegression(random_state = 12345).fit(features_train,target_train)
predictions_regression = model_regression.predict(features_valid)
score_regression = accuracy_score(target_valid,predictions_regression)
print(f'Показатель accuracy модели логистической регрессии равен {score_regression}')

Показатель accuracy модели логистической регрессии равен 0.7107309486780715


Таким образом, среди всех моделей наибольший показатель accuracy у модели случайного леса с 8 деревьями и показателем depth 8

## Проверка модели на тестовой выборке

In [15]:
# Проверим лучшую модель на тестовой выборке
predictions_test = best_model_forest.predict(features_test)
accuracy_test = accuracy_score(target_test,predictions_test)
print(f'Показатель accuracy по тестовой выборке у наилучшей модели: {accuracy_test}')

Показатель accuracy по тестовой выборке у наилучшей модели: 0.7962674961119751
